In [1]:
!pip install pandas
!pip install dowhy
!pip install pgmpy
!pip install pygraphviz
!pip install numpy
!pip install graphviz
!pip install numpy
!pip install networkx
!pip install causal-learn

In [3]:
import pandas as pd
import os
path = os.getcwd()+"/local/uni/inference/Task2/"

print()
file_path = path + './social_media_engagement_data.csv'

# Load data from a csv-file into a DataFrame. The header parameter is set to True if your csv-file contains column names in the first row.
df = pd.read_csv(file_path, header=0)

print(df)

 


       Platform  Post Type  Likes  Comments  Shares  Impressions  Reach
0             1          1    415        77      85         2619   1842
1             2          1    908       125      85         1223   3884
2             2          1    412       419      58         2165    849
3             3          2    667       153      29         7191   3519
4             4          3    659       215      34         3312   2032
...         ...        ...    ...       ...     ...          ...    ...
49995         1          2    488       469      10         2405   1183
49996         4          2    764        96     108         6207   4059
49997         2          1    641       163     198         8605   1918
49998         2          1      0       465     188         6122   2399
49999         4          2    640       175     185         9564   1871

[50000 rows x 7 columns]


In [30]:
# Create Model
from pgmpy.models import BayesianNetwork

model = BayesianNetwork([('Shares', 'Impressions'), ('Likes', 'Impressions'), ('Comments', 'Impressions'), ('Reach', 'Impressions'), ('Post Type', 'Comments'), ('Post Type', 'Impressions'), ('Platform', 'Post Type')])
viz = model.to_graphviz()

viz.draw(path + "./test.png", prog="dot")

In [38]:

from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import JunctionTree
cpd = MaximumLikelihoodEstimator(model, df).estimate_cpd('Post Type')
print(cpd)

+------------------+-----+---------------------+
| Platform         | ... | Platform(Twitter)   |
+------------------+-----+---------------------+
| Post Type(Image) | ... | 0.33426073131955486 |
+------------------+-----+---------------------+
| Post Type(Link)  | ... | 0.33306836248012717 |
+------------------+-----+---------------------+
| Post Type(Video) | ... | 0.332670906200318   |
+------------------+-----+---------------------+


In [5]:
from causallearn.search.ConstraintBased.PC import pc
from causallearn.search.ScoreBased.GES import ges
from causallearn.utils.cit import kci
labels = [f'{col}' for i, col in enumerate(df.columns)]
data = df.to_numpy()

print(data)

cg = pc(data, 0.05, kci, kernelZ='Polynomial', approx=False, est_width='median')
from causallearn.utils.GraphUtils import GraphUtils
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import io

pyd = GraphUtils.to_pydot(cg.G, labels=labels)
tmp_png = pyd.create_png(f="png")
fp = io.BytesIO(tmp_png)
img = mpimg.imread(fp, format='png')
plt.axis('off')
plt.imshow(img)
plt.show()

[[   1    1  415 ...   85 2619 1842]
 [   2    1  908 ...   85 1223 3884]
 [   2    1  412 ...   58 2165  849]
 ...
 [   2    1  641 ...  198 8605 1918]
 [   2    1    0 ...  188 6122 2399]
 [   4    2  640 ...  185 9564 1871]]


  0%|          | 0/7 [00:00<?, ?it/s]